In [1]:
import json
import random
from datetime import datetime, timedelta

In [2]:
from ipywidgets import Layout, widgets

In [3]:
from ipyleaflet import Map, GeoJSON, WidgetControl

In [4]:
from branca.colormap import linear

In [5]:
cmap = linear.YlOrRd_09.scale(0, 150)

In [6]:
from load_data import load_dataframe_dep

df = load_dataframe_dep()
df = df[(df.cl_age90 == 0)].copy()

In [7]:
date_max = df.index.max()
date_max_obj = datetime.strptime(date_max, "%Y-%m-%d")
date_m7 = date_max_obj - timedelta(7)
date_m7 = date_m7.strftime("%Y-%m-%d")
date_m7

'2020-09-08'

In [8]:
df7 = df[(df.index > date_m7)][["dep", "P"]]

In [9]:
df7 = df7.groupby("dep").sum() 

In [10]:
from population import population

In [11]:
incidence = {}

for dep in df7.index:
    if dep in population:
        incidence[dep] = 100000/population[dep] * df7.P.loc[dep]
    else:
        incidence[dep] = 0
        


In [12]:
df7["incidence"] = incidence.values()

In [13]:
tmp = df[(df.index == date_max)].copy()

In [14]:
tmp["ratio"] = 100 * tmp["P"] / tmp["T"]

In [15]:
tmp[tmp.dep == "01"].ratio.values[0]

8.365019011406844

In [16]:
with open('data/departements-version-simplifiee.geojson', 'r') as f:
    data = json.load(f)


In [17]:
def get_positivity(dep):
    return tmp[tmp.dep == dep].ratio.values[0]

In [18]:
def get_incidence(dep):
    return df7.incidence.loc[dep]

In [19]:
get_incidence("01")

77.25188466620037

In [20]:
def color_dep(feature):
    dep = feature["properties"]["code"]
    value = get_incidence(dep)
    color = cmap(value)    
    return {
        'color': 'black',
        'fillColor': color,
    }

In [21]:
m = Map(center=(47.5, 2), zoom=6,
        layout=Layout(width='100%', height='670px'))

geo_json = GeoJSON(
    data=data,
    style={
        'opacity': 1, 'dashArray': '0', 'fillOpacity': 0.6, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    style_callback=color_dep
)
m.add_layer(geo_json)

out = widgets.Output(layout={'border': '1px solid black'})
with out:
    display(cmap)
widget_control = WidgetControl(widget=out, position='topright')
m.add_control(widget_control)
m

Map(center=[47.5, 2], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…